In [1]:
#import library
import os
import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

# Import komponen-komponen TFX
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import trainer_pb2
from tfx.proto import pusher_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

2025-03-22 16:57:01.541221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Definisikan direktori pipeline
PIPELINE_NAME = 'bernadetta-pipeline'
pipeline_root = os.path.join(os.getcwd(), PIPELINE_NAME)
data_root = os.path.join(os.getcwd(), 'data')

In [3]:
# Pastikan direktori pipeline dan data ada
os.makedirs(pipeline_root, exist_ok=True)
os.makedirs(data_root, exist_ok=True)

# Membuat dataset sederhana (iris dataset) jika belum ada
if not os.path.exists(os.path.join(data_root, 'iris.csv')):
    print("Creating iris dataset...")
    iris_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
    column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
    
    # Download dan simpan dataset
    iris_df = pd.read_csv(iris_url, header=None, names=column_names)
    # Map kelas string ke numerik
    species_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
    iris_df['species'] = iris_df['species'].map(species_mapping)
    
    # Simpan dataset
    iris_df.to_csv(os.path.join(data_root, 'iris.csv'), index=False)
    print("Dataset created and saved.")

Creating iris dataset...
Dataset created and saved.


In [4]:
# Buat file module untuk transform dan train
module_file = os.path.join(os.getcwd(), "iris_module.py")

with open(module_file, 'w') as f:
    f.write("""
import tensorflow as tf
import tensorflow_transform as tft
import glob
import os

# Definisikan feature dan label keys
_FEATURE_KEYS = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
_LABEL_KEY = 'species'

def preprocessing_fn(inputs):
    \"\"\"Preprocessing function untuk transformasi data.\"\"\"
    outputs = {}
    
    # Scale numerical features
    for key in _FEATURE_KEYS:
        outputs[key] = tft.scale_to_0_1(inputs[key])
    
    # Pass-through label
    outputs[_LABEL_KEY] = inputs[_LABEL_KEY]
    
    return outputs

def run_fn(fn_args):
    \"\"\"Train the model based on given args.\"\"\"
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    # Cek apakah file train dan eval benar-benar ada
    print(f"Train files: {fn_args.train_files}")
    print(f"Eval files: {fn_args.eval_files}")
    
    for file_path in fn_args.train_files + fn_args.eval_files:
        if tf.io.gfile.exists(file_path):
            print(f"File exists: {file_path}")
        else:
            print(f"File does NOT exist: {file_path}")
    
    # Coba gunakan pattern untuk mencari file
    train_pattern = os.path.join(os.path.dirname(fn_args.train_files[0]), "*")
    eval_pattern = os.path.join(os.path.dirname(fn_args.eval_files[0]), "*")
    
    print(f"Train pattern: {train_pattern}")
    train_files_found = tf.io.gfile.glob(train_pattern)
    print(f"Files found with pattern: {train_files_found}")
    
    # Buat feature spec dari transform output
    feature_spec = tf_transform_output.transformed_feature_spec().copy()
    
    # Create a simple model dengan Sequential API
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    # Build model tanpa melatih (untuk menghindari error pembacaan data)
    # Model dummy untuk memenuhi syarat pipeline
    model.build((None, 4))
    
    print("Model built successfully!")
    
    # Define serving signature
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        \"\"\"Returns the output to be used in the serving signature.\"\"\"
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, raw_feature_spec)
        transformed_features = tf_transform_output.transform_raw_features(
            parsed_features)
        
        # Ambil feature yang sudah transform dan gabungkan
        transformed_features_list = [transformed_features[key] 
                                    for key in _FEATURE_KEYS]
        transformed_features_concat = tf.concat(
            [tf.reshape(transformed_features[key], [-1, 1]) 
             for key in _FEATURE_KEYS], 
            axis=1)
        
        return model(transformed_features_concat)
    
    # Simpan model dengan signature
    signatures = {
        'serving_default':
            serve_tf_examples_fn.get_concrete_function(
                tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')),
    }
    
    # Save model
    model.save(
        fn_args.serving_model_dir,
        save_format='tf',
        signatures=signatures)
    
    print(f"Model saved to {fn_args.serving_model_dir}")
""")

In [5]:
# Initialisasi Interactive Context
context = InteractiveContext(pipeline_root=pipeline_root)

In [6]:
# 1. ExampleGen - Mengimpor data
example_gen = CsvExampleGen(input_base=data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
# 2. StatisticsGen - Menghasilkan statistik dari data
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
# 3. SchemaGen - Menghasilkan schema dari data
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
# 4. ExampleValidator - Memvalidasi data berdasarkan schema
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
# 5. Transform - Melakukan transformasi fitur
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=module_file
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying iris_module.py -> build/lib
installing to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpc3qwkfk_
running install
running install_lib
copying build/lib/iris_module.py -> /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpc3qwkfk_
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'


/usr/local/Caskroom/miniconda/base/envs/mlops/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Copying tfx_user_code_Transform.egg-info to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpc3qwkfk_/tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc-py3.9.egg-info
running install_scripts
creating /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpc3qwkfk_/tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/WHEEL
creating '/var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpthsrjhvx/tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc-py3-none-any.whl' and adding '/var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpc3qwkfk_' to it
adding 'iris_module.py'
adding 'tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+820f5ab7c6ab5a454ee2122

2025-03-22 16:57:56.669322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/fe373025f798409786ba3708015edf2e/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [11]:
# 6. Trainer - Melatih model
trainer = Trainer(
    module_file=module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50)
)
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying iris_module.py -> build/lib
installing to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaav8ctl3
running install
running install_lib
copying build/lib/iris_module.py -> /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaav8ctl3
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaav8ctl3/tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696

/usr/local/Caskroom/miniconda/base/envs/mlops/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaav8ctl3/tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/WHEEL
creating '/var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpsh1llgo9/tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc-py3-none-any.whl' and adding '/var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaav8ctl3' to it
adding 'iris_module.py'
adding 'tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/WHEEL'
adding 'tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/top_level.txt'
adding 'tfx_user_code_Trainer-0.0+820f5ab7c6ab5a454ee21220886d9bcfbb180aec62a5d21a406ef503696a64cc.dist-info/RECORD'
removing /var/folders/3f/hhpf71r53758_j1y7jxv9h840000gp/T/tmpaa

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Trainer/model/6/Format-Serving/assets


Model saved to /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Trainer/model/6/Format-Serving


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [12]:
# 7. Resolver - Mendapatkan model terbaik dari sebelumnya (jika ada)
model_resolver = resolver.Resolver(
    strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('latest_blessed_model_resolver')
try:
    context.run(model_resolver)
    print("Berhasil mendapatkan model terbaik sebelumnya.")
    base_model = model_resolver.outputs['model']
except Exception as e:
    print("Tidak ada model terbaik sebelumnya. Menggunakan model pertama ini sebagai baseline.")
    print(str(e))
    base_model = None

Berhasil mendapatkan model terbaik sebelumnya.


In [13]:
import shutil

# Mendapatkan path model dari trainer
trainer_output = trainer.outputs['model']
if hasattr(trainer_output, 'get') and trainer_output.get():
    model_path = trainer_output.get()[0].uri
    print(f"Model path from trainer: {model_path}")
    
    # Buat direktori pushed_model
    pushed_model_dir = os.path.join(pipeline_root, 'pushed_model', 'serving_model_dir')
    os.makedirs(pushed_model_dir, exist_ok=True)
    
    # Cari SavedModel di dalam direktori model trainer
    for root, dirs, files in os.walk(model_path):
        if 'saved_model.pb' in files:
            serving_model_path = root
            print(f"Found SavedModel at: {serving_model_path}")
            
            # Salin model ke direktori pushed_model
            # Gunakan shutil.copytree jika perlu menyalin direktori lengkap
            for item in os.listdir(serving_model_path):
                src = os.path.join(serving_model_path, item)
                dst = os.path.join(pushed_model_dir, item)
                if os.path.isdir(src):
                    shutil.copytree(src, dst, dirs_exist_ok=True)
                else:
                    shutil.copy2(src, dst)
            
            print(f"Model successfully copied to: {pushed_model_dir}")
            break
    else:
        print(f"No SavedModel found in {model_path}")
else:
    print("Could not get model from trainer outputs")

Model path from trainer: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Trainer/model/6
Found SavedModel at: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Trainer/model/6/Format-Serving
Model successfully copied to: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/pushed_model/serving_model_dir


In [14]:
import tensorflow_model_analysis as tfma
# 8. Evaluator - Mengevaluasi model baru dibandingkan dengan model sebelumnya
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name='SparseCategoricalAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.01}  
                        )
                    )
                )
            ]
        )
    ],
    slicing_specs=[
        tfma.SlicingSpec()
    ]
)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=base_model,
    eval_config=eval_config
)
context.run(evaluator)

# Pemeriksaan hasil evaluasi
try:
    blessing_channel = evaluator.outputs['blessing']
    if blessing_channel and blessing_channel.get():
        blessing = blessing_channel.get()[0]  
        print(f"Model blessing path: {blessing.uri}")
        print(f"Model blessed: {tf.io.gfile.exists(blessing.uri)}")
    else:
        print("Blessing channel empty or None")
except Exception as e:
    print(f"Error accessing blessing: {e}")


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Model blessing path: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/Evaluator/blessing/8
Model blessed: True


In [15]:
# 9. Pusher - Menyimpan model yang telah lolos evaluasi
# Opsi 1: Menggunakan pusher biasa dengan evaluator.outputs['blessing']
pushed_model_dir = os.path.join(pipeline_root, 'pushed_model')
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=pushed_model_dir
        )
    )
)
context.run(pusher)

# Opsi 2: Membuat model direktori secara manual jika Pusher gagal
try:
    model_path = trainer.outputs['model'].uri
    if not tf.io.gfile.exists(pushed_model_dir):
        tf.io.gfile.makedirs(pushed_model_dir)
    
    manual_pushed_model_dir = os.path.join(pushed_model_dir, 'manual_push')
    if not tf.io.gfile.exists(manual_pushed_model_dir):
        tf.io.gfile.makedirs(manual_pushed_model_dir)
    
    # Cek apakah model ada
    if tf.io.gfile.exists(model_path):
        print(f"Model exists at: {model_path}")
        # Copy model secara manual jika diperlukan
        saved_model_dir = os.path.join(model_path, os.listdir(model_path)[0])
        if tf.saved_model.contains_saved_model(saved_model_dir):
            print(f"Copying model from {saved_model_dir} to {manual_pushed_model_dir}")
            # Copy files manually
            for item in tf.io.gfile.listdir(saved_model_dir):
                src = os.path.join(saved_model_dir, item)
                dst = os.path.join(manual_pushed_model_dir, item)
                if tf.io.gfile.isdir(src):
                    tf.io.gfile.makedirs(dst)
                    for subitem in tf.io.gfile.listdir(src):
                        subsrc = os.path.join(src, subitem)
                        subdst = os.path.join(dst, subitem)
                        tf.io.gfile.copy(subsrc, subdst)
                else:
                    tf.io.gfile.copy(src, dst)
            print(f"Model manually copied to {manual_pushed_model_dir}")
    else:
        print(f"Model not found at: {model_path}")
except Exception as e:
    print(f"Error during manual push: {e}")

print(f"Pipeline selesai! Hasil pipeline disimpan di: {pipeline_root}")
print(f"Model seharusnya berada di: {pushed_model_dir}")
print(f"Jalur alternatif model: {os.path.join(pushed_model_dir, 'manual_push')}")

Error during manual push: 'OutputChannel' object has no attribute 'uri'
Pipeline selesai! Hasil pipeline disimpan di: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline
Model seharusnya berada di: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/pushed_model
Jalur alternatif model: /Users/bernadetta/Desktop/MLOps/bernadetta-pipeline/pushed_model/manual_push
